# Advent of code 2022 - 07

In [1]:
import re

from utils import read_txt_file

## Code

In [24]:
class FileSystem:
    def __init__(self, debug: bool = False):
        self.debug = debug

        self.current_dir = "/"
        self.parents = {"/": None}
        self.children = {"/": []}
        self.total_file_sizes = {"/": 0}  # Only files in this diretory

        self.total_dir_sizes = dict()  # All files and subdiretories

    def execute(self, command: str):
        if self.debug:
            print(f"Executing >> '{command}'")

        if command[:2] == "cd":
            self.run_change_directory(command)
        elif command[:2] == "ls":
            self.run_list(command)
        else:
            raise ValueError(f"Wrong command prefix: '{command[:2]}'")

    def run_change_directory(self, command):
        move_to = re.search("(?<=cd )\S+", command).group(0)
        if move_to == "..":
            self.current_dir = self.parents[self.current_dir]
        elif move_to == "/":
            self.current_dir = "/"
        else:
            if self.current_dir == "/":
                self.current_dir = f"/{move_to}"
            else:
                self.current_dir = f"{self.current_dir}/{move_to}"

        if self.debug:
            print(f"-> Changed to new directory: {self.current_dir}")

    def _search_pattern(self, which: str, command: str):
        if which == "dir":
            pattern = re.compile("(?<=dir )\S+")
        elif which == "file":
            pattern = re.compile("^\d+")
        else:
            raise ValueError(f"Wrong value for which: '{which}'")

        try:
            return pattern.search(command).group(0)

        except AttributeError:
            return None

    def _add_dir_relationship(self, parent: str, child: str):
        if parent == "/":
            child_full = f"/{child}"
        else:
            child_full = f"{self.current_dir}/{child}"

        self.parents[child_full] = parent

        self.children[parent].append(child_full)
        self.children[parent] = list(set(self.children[parent]))  # drop duplicates !

        if self.debug:
            print(f"-> parent of '{child_full}': '{self.parents[child_full]}'")
            print(f"-> chlidren of '{parent}': '{self.children[parent]}'")

    def _add_total_file_sizes(self, parent: str, new_file_size: str):
        self.total_file_sizes[parent] += int(new_file_size)

    def run_list(self, command):
        # Split line results empty commands
        line_results = command.split("\n")[1:]
        try:
            line_results.remove("")
            if self.debug:
                print(f"REMOVED line in  '{command}'\n")

        except ValueError:
            pass

        # Initalisation
        self.children[self.current_dir] = []
        self.total_file_sizes[self.current_dir] = 0

        # Data capture
        for f in line_results:
            dir_pattern = self._search_pattern(which="dir", command=f)
            file_pattern = self._search_pattern(which="file", command=f)
            # print(f"{f}: dir pattern found: '{dir_pattern}'")
            # print(f"{f}: file pattern found: '{file_pattern}'")
            if dir_pattern:
                self._add_dir_relationship(parent=self.current_dir, child=dir_pattern)
            elif file_pattern:
                self._add_total_file_sizes(
                    parent=self.current_dir, new_file_size=file_pattern
                )
            else:
                raise ValueError(f"No pattern found in '{f}'")

        if self.debug:
            print(
                f"-> total file size of '{self.current_dir}': '{self.total_file_sizes[self.current_dir]}'"
            )

    def compute_total_dir_size(self, dir_name: str):
        """Get file sizes recursively!"""
        if len(self.children[dir_name]) == 0:
            self.total_dir_sizes[dir_name] = self.total_file_sizes[dir_name]
            return self.total_file_sizes[dir_name]
        else:
            total_dir_size = self.total_file_sizes[dir_name]
            for child_dir in self.children[dir_name]:
                subdir_size = self.compute_total_dir_size(child_dir)
                total_dir_size += subdir_size

            self.total_dir_sizes[dir_name] = total_dir_size
            return total_dir_size


def solve_07(path: str, is_part_2: bool = False, debug: bool = False):
    input_txt = read_txt_file(path)

    fs = FileSystem(debug=debug)
    commands = [x.replace("$ ", "") for x in input_txt.split("\n$ ")]

    for i, c in enumerate(commands):
        fs.execute(c)
        if debug:
            print()

    if debug:
        print(f"parents: {fs.parents}")
        print(f"children: {fs.children}")
        print(f"total_file_sizes: {fs.total_file_sizes}")

    _ = fs.compute_total_dir_size("/")

    if debug:
        print(f"\ntotal_dir_sizes: {fs.total_dir_sizes}")

    if not is_part_2:
        return sum([x for x in fs.total_dir_sizes.values() if x <= 100_000])

    else:
        filesystem_size = 70_000_000
        space_required = 30_000_000
        tot_file_size = fs.total_dir_sizes["/"]

        adequate_dir_to_delete = dict()
        # print(fs.total_dir_sizes)
        for d, s in fs.total_dir_sizes.items():
            if (filesystem_size - tot_file_size + s) >= space_required:
                adequate_dir_to_delete[d] = s

        if debug:
            print(adequate_dir_to_delete)

        return min([x for x in adequate_dir_to_delete.values()])


# Run some tests
example_file = f"inputs/07_example.txt"
assert solve_07(example_file, debug=False) == 95437
assert solve_07(example_file, is_part_2=True, debug=False) == 24933642

input_file = "inputs/07_input.txt"
assert solve_07(input_file, debug=False) == 1778099



## Example

In [3]:
example_file = "inputs/07_example.txt"

In [4]:
solve_07(example_file, debug=False)

95437

In [25]:
solve_07(example_file, is_part_2=True, debug=False)

24933642

## Puzzle

In [26]:
input_file = "inputs/07_input.txt"

In [27]:
solve_07(input_file, debug=False)

1778099

In [28]:
solve_07(input_file, is_part_2=True, debug=False)

1623571